In [6]:
import sys
!{sys.executable} -m pip install folium
import folium
!{sys.executable} -m pip install pandas
import pandas as pd
!{sys.executable} -m pip install numpy
import numpy as np
!{sys.executable} -m pip install seaborn
import seaborn as sns
import matplotlib.pyplot as plt
!{sys.executable} -m pip install matplotlib.pyplot

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement matplotlib.pyplot (from ver

In [8]:
pop_data = pd.read_csv("population_by_country_2020.csv")
pop_data.head()
pop_data.at[2,'Country (or dependency)']='US'
data_raw = pd.read_csv("12-25-2021_denmark.csv", sep=";")
pop_data.set_index("Country (or dependency)", inplace = True)
pop_data.head()


,Population (2020),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share
Country (or dependency),,,,,,,,,,
China,1440297825,0.39 %,5540090,153,9388211,-348399.0,1.7,38,61 %,18.47 %
India,1382345085,0.99 %,13586631,464,2973190,-532687.0,2.2,28,35 %,17.70 %
US,331341050,0.59 %,1937734,36,9147420,954806.0,1.8,38,83 %,4.25 %
Indonesia,274021604,1.07 %,2898047,151,1811570,-98955.0,2.3,30,56 %,3.51 %
Pakistan,221612785,2.00 %,4327022,287,770880,-233379.0,3.6,23,35 %,2.83 %


In [9]:
pop_dropped = pop_data.drop(["Yearly Change","Net Change","Density (P/Km²)","Land Area (Km²)","Migrants (net)","Fert. Rate","Med. Age","Urban Pop %","World Share"], axis = 1)
pop_dropped.round(0).astype(int)

,Population (2020)
Country (or dependency),
China,1440297825
India,1382345085
US,331341050
Indonesia,274021604
Pakistan,221612785
...,...
Montserrat,4993
Falkland Islands,3497
Niue,1628


In [10]:
data_test = data_raw[['Country_Region','Lat','Long_', 'Confirmed', 'Deaths']].copy()

data_lt_lg = data_test[['Country_Region','Lat','Long_']].copy()
data_lt_lg.drop_duplicates(subset = 'Country_Region', keep = 'first', inplace = True)
data_lt_lg.set_index('Country_Region', inplace=True)

data_test.drop(['Lat', 'Long_'], axis =1 , inplace = True)
data_test = data_test.groupby('Country_Region').sum()
data_test.round(0).astype(int)
data_lt_lg.head()
data_lt_lg.shape

(196, 2)

In [11]:
data_test.head()
data_test.shape


(196, 2)

In [12]:
data_final = data_lt_lg.join(data_test, how = 'inner')
data_final.drop(['Diamond Princess', 'MS Zaandam'], inplace = True)
data_final.shape
data_new_final = pd.concat([data_final, pop_dropped], axis=1, join="inner")
data_new_final["Mortality"] = data_new_final["Deaths"]*100000/data_new_final["Population (2020)"]
data_new_final = data_new_final.round({'Mortality': 0})
data_new_final["main_data"] = data_new_final.index + "\n" + "\nConfirmed cases: " + data_new_final["Confirmed"].astype(str) + " \nDeaths: " + data_new_final["Deaths"].astype(str) + "\nMortality per 100000 population: " + data_new_final["Mortality"].astype(str)
data_new_final.at["Russia",'Lat']= 55.7504461
data_new_final.at["Russia",'Long_']= 37.6174943


In [13]:
data_new_final.loc['US']


Lat                                                          32.539527
Long_                                                       -86.644082
Confirmed                                                     52098906
Deaths                                                          816533
Population (2020)                                            331341050
Mortality                                                        246.0
main_data            US\n\nConfirmed cases: 52098906 \nDeaths: 8165...
Name: US, dtype: object

In [14]:
my_map = folium.Map(
    location = [52.520008,13.404954],
    zoom_start = 4
)
for index,country in data_new_final.iterrows():
    # ind = data_merg.index[country]
    folium.Marker(
    location = [country["Lat"],country["Long_"]],
    popup = country["main_data"],
    tooltip = index
    ).add_to(my_map)
   

my_map

In [15]:
data_final.loc['Canada']


Lat          5.393330e+01
Long_       -1.165765e+02
Confirmed    1.987909e+06
Deaths       3.020300e+04
Name: Canada, dtype: float64

In [17]:
my_map.save('map.html')